# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it.

You will find more info about virtual environments in the course content and on the web.

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [437]:
# import the requests library (1 line)
import requests
# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url = 'https://country-leaders.onrender.com'
# assign the /status endpoint to another variable called status_url (1 line)
status_url = 'https://country-leaders.onrender.com/status/'
# query the /status endpoint using the get() method and store it in the req variable (1 line)
r = requests.get(status_url)
# check the status_code using a condition and print appropriate messages (4 lines)
status_code = r.status_code
if status_code == 200:
    print(f'Text response of status code: {r.reason}')
else:
    print(f'Status code: {status_code}')

Text response of status code: OK


### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the Requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [438]:
# Set the countries_url variable (1 line)
countries_url = 'https://country-leaders.onrender.com/countries'
# query the /countries endpoint using the get() method and store it in the req variable (1 line)
r = requests.get(countries_url)
# Get the JSON content and store it in the countries variable (1 line)
countries = r.json()
# display the request's status code and the countries variable (1 line)
print(countries, r.status_code)

{'message': 'The cookie is missing'} 403


### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [439]:
# Set the cookie_url variable (1 line)
cookie_url = 'https://country-leaders.onrender.com/cookie'
# Query the enpoint, set the cookies variable and display it (2 lines)
response_cookie = requests.get(cookie_url)
cookies = response_cookie.cookies # Returns a dictionary of cookies
print(cookies)

<RequestsCookieJar[<Cookie user_cookie=540b8f95-e24f-434a-8520-df11d8388a3c for country-leaders.onrender.com/>]>


Try to query the countries endpoint using the cookie, save the output and print it.

In [440]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
countries = requests.get(countries_url, cookies=cookies)
countries_list = countries.json()
# display the countries variable (1 line)
print(countries, countries.reason)
print(countries.content)
print(f"countries list: {countries_list}")

<Response [200]> OK
b'["be","fr","ma","ru","us"]'
countries list: ['be', 'fr', 'ma', 'ru', 'us']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### Getting the actual data from the API

Query the `/leaders` endpoint.

In [441]:
# Set the leaders_url variable (1 line)
leaders_url = 'https://country-leaders.onrender.com/leaders'
# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders = requests.get(leaders_url, cookies=cookies)
# display the leaders variable (1 line)
print(leaders.text)

{"message":"Please specify a country"}


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [442]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
params = {
    "country": "be"
}
# assign the output to the leaders variable (1 line)
leaders = requests.get(leaders_url, params=params, cookies=cookies)
print(leaders, leaders.json())
# display the leaders variable (1 line)


<Response [200]> [{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'death_date': None, 'place_of_birth': 'Wervik', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'start_mandate': '2009-11-25', 'end_mandate': '2011-12-06'}, {'id': 'Q12983', 'first_name': 'Herman', 'last_name': 'None', 'birth_date': '1947-10-31', 'death_date': None, 'place_of_birth': 'Etterbeek', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy', 'start_mandate': '2008-12-30', 'end_mandate': '2009-11-25'}, {'id': 'Q14989', 'first_name': 'Léon', 'last_name': 'Delacroix', 'birth_date': '1867-12-27', 'death_date': '1929-10-15', 'place_of_birth': 'Saint-Josse-ten-Noode', 'wikipedia_

### A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [443]:
# 4 lines
leaders_per_country = {}
for country in countries_list:
    print(country)
    params = {
        "country": country
    }
    leaders_response = requests.get(leaders_url, params=params, cookies=cookies)
    print(leaders_response.json())
    leaders_per_country[country] = leaders_response.json()
print(f"Leaders per country dict: {leaders_per_country}")

be
[{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'death_date': None, 'place_of_birth': 'Wervik', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'start_mandate': '2009-11-25', 'end_mandate': '2011-12-06'}, {'id': 'Q12983', 'first_name': 'Herman', 'last_name': 'None', 'birth_date': '1947-10-31', 'death_date': None, 'place_of_birth': 'Etterbeek', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy', 'start_mandate': '2008-12-30', 'end_mandate': '2009-11-25'}, {'id': 'Q14989', 'first_name': 'Léon', 'last_name': 'Delacroix', 'birth_date': '1867-12-27', 'death_date': '1929-10-15', 'place_of_birth': 'Saint-Josse-ten-Noode', 'wikipedia_url': 'https:/

In [444]:
# or 1 line
leaders_per_country = {x : requests.get(leaders_url, params={ "country" : x}, cookies=cookies).json() for x in countries_list}
print(leaders_per_country)

{'be': [{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'death_date': None, 'place_of_birth': 'Wervik', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'start_mandate': '2009-11-25', 'end_mandate': '2011-12-06'}, {'id': 'Q12983', 'first_name': 'Herman', 'last_name': 'None', 'birth_date': '1947-10-31', 'death_date': None, 'place_of_birth': 'Etterbeek', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy', 'start_mandate': '2008-12-30', 'end_mandate': '2009-11-25'}, {'id': 'Q14989', 'first_name': 'Léon', 'last_name': 'Delacroix', 'birth_date': '1867-12-27', 'death_date': '1929-10-15', 'place_of_birth': 'Saint-Josse-ten-Noode', 'wikipedia_url': 'htt

It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [445]:
# < 15 lines
def get_leaders():
    root_url = 'https://country-leaders.onrender.com'
    cookies = requests.get(f"{root_url}/cookie").cookies  # Getting the cookies
    countries = requests.get(f"{root_url}/countries", cookies=cookies).json()
    leaders_per_country = {country : requests.get(f"{root_url}/leaders", {"country": country}, cookies=cookies).json() for country in countries}
    return leaders_per_country

Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [446]:
# 2 lines
print(get_leaders())

{'be': [{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'death_date': None, 'place_of_birth': 'Wervik', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'start_mandate': '2009-11-25', 'end_mandate': '2011-12-06'}, {'id': 'Q12983', 'first_name': 'Herman', 'last_name': 'None', 'birth_date': '1947-10-31', 'death_date': None, 'place_of_birth': 'Etterbeek', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy', 'start_mandate': '2008-12-30', 'end_mandate': '2009-11-25'}, {'id': 'Q14989', 'first_name': 'Léon', 'last_name': 'Delacroix', 'birth_date': '1867-12-27', 'death_date': '1929-10-15', 'place_of_birth': 'Saint-Josse-ten-Noode', 'wikipedia_url': 'htt

## Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [447]:
# 3 lines
leaders_data = get_leaders()
wiki_url_ru = leaders_data['ru'][0]['wikipedia_url']
# print(wiki_url_ru)
for country, leaders in leaders_data.items():
    for leader in leaders:
        all_leaders_wiki = leader['wikipedia_url']
        print(all_leaders_wiki)


https://nl.wikipedia.org/wiki/Guy_Verhofstadt
https://nl.wikipedia.org/wiki/Yves_Leterme
https://nl.wikipedia.org/wiki/Herman_Van_Rompuy
https://nl.wikipedia.org/wiki/L%C3%A9on_Delacroix
https://nl.wikipedia.org/wiki/Henri_Carton_de_Wiart
https://nl.wikipedia.org/wiki/Georges_Theunis
https://nl.wikipedia.org/wiki/Aloys_Van_de_Vyvere
https://nl.wikipedia.org/wiki/Prosper_Poullet
https://nl.wikipedia.org/wiki/Henri_Jaspar
https://nl.wikipedia.org/wiki/Jules_Renkin
https://nl.wikipedia.org/wiki/Paul_van_Zeeland
https://nl.wikipedia.org/wiki/Achiel_Van_Acker
https://nl.wikipedia.org/wiki/Camille_Huysmans
https://nl.wikipedia.org/wiki/Gaston_Eyskens
https://nl.wikipedia.org/wiki/Leo_Tindemans
https://nl.wikipedia.org/wiki/Elio_Di_Rupo
https://nl.wikipedia.org/wiki/Jean-Luc_Dehaene
https://nl.wikipedia.org/wiki/Frans_Schollaert
https://nl.wikipedia.org/wiki/Pieter_de_Decker
https://nl.wikipedia.org/wiki/Paul-Henri_Spaak
https://nl.wikipedia.org/wiki/Auguste_Beernaert
https://nl.wikipedia.org

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [448]:
# 3 lines
from bs4 import BeautifulSoup
# Send a GET request to the Wikipedia page and print out the raw HTML text
response = requests.get(all_leaders_wiki).text
# print(response)
soup = BeautifulSoup(response, 'html.parser')
# print(soup.prettify()) # Prettify is to make the html nmore readable
print(soup.get_text())






Calvin Coolidge - Wikipedia



































Jump to content







Main menu





Main menu
move to sidebar
hide



		Navigation
	


Main pageContentsCurrent eventsRandom articleAbout WikipediaContact us





		Contribute
	


HelpLearn to editCommunity portalRecent changesUpload file



















Search











Search















Donate








Appearance
















Create account

Log in








Personal tools





 Create account Log in





		Pages for logged out editors learn more



ContributionsTalk




























Contents
move to sidebar
hide




(Top)





1
Early life and family history








2
Early career and marriage




Toggle Early career and marriage subsection





2.1
Education and law practice








2.2
Marriage and family










3
Local political office (1898−1915)




Toggle Local political office (1898−1915) subsection





3.1
City offices








3.2
Massachusetts state legislator and mayor










4
L

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [449]:
# 2 lines
paragraphs = soup.find_all('p')
print(paragraphs)
for paragraph in paragraphs:  # Need to apply prettify() to individual elements within the list, not to the entire list itself.
    if paragraph.find('b'):
         paragraph_text = {
              'paragraph_text': paragraph.text.strip(),
              'bold_text': [b.text.strip() for b in paragraph.find_all('b')]
         }
     #     print(paragraph_text) # Print the extracted information


[<p class="mw-empty-elt">
</p>, <p><b>Calvin Coolidge</b> (born <b>John Calvin Coolidge Jr.</b><sup class="reference" id="cite_ref-2"><a href="#cite_note-2"><span class="cite-bracket">[</span>1<span class="cite-bracket">]</span></a></sup> <span class="rt-commentedText nowrap"><span class="IPA nopopups noexcerpt" lang="en-fonipa"><a href="/wiki/Help:IPA/English" title="Help:IPA/English">/<span style="border-bottom:1px dotted"><span title="/ˈ/: primary stress follows">ˈ</span><span title="'k' in 'kind'">k</span><span title="/uː/: 'oo' in 'goose'">uː</span><span title="'l' in 'lie'">l</span><span title="/ɪ/: 'i' in 'kit'">ɪ</span><span title="/dʒ/: 'j' in 'jam'">dʒ</span></span>/</a></span></span> <a href="/wiki/Help:Pronunciation_respelling_key" title="Help:Pronunciation respelling key"><i title="English pronunciation respelling"><span style="font-size:90%">KOOL</span>-ij</i></a>; July 4, 1872 – January 5, 1933) was the 30th <a href="/wiki/President_of_the_United_States" title="President

In [450]:
print(paragraphs)

[<p class="mw-empty-elt">
</p>, <p><b>Calvin Coolidge</b> (born <b>John Calvin Coolidge Jr.</b><sup class="reference" id="cite_ref-2"><a href="#cite_note-2"><span class="cite-bracket">[</span>1<span class="cite-bracket">]</span></a></sup> <span class="rt-commentedText nowrap"><span class="IPA nopopups noexcerpt" lang="en-fonipa"><a href="/wiki/Help:IPA/English" title="Help:IPA/English">/<span style="border-bottom:1px dotted"><span title="/ˈ/: primary stress follows">ˈ</span><span title="'k' in 'kind'">k</span><span title="/uː/: 'oo' in 'goose'">uː</span><span title="'l' in 'lie'">l</span><span title="/ɪ/: 'i' in 'kit'">ɪ</span><span title="/dʒ/: 'j' in 'jam'">dʒ</span></span>/</a></span></span> <a href="/wiki/Help:Pronunciation_respelling_key" title="Help:Pronunciation respelling key"><i title="English pronunciation respelling"><span style="font-size:90%">KOOL</span>-ij</i></a>; July 4, 1872 – January 5, 1933) was the 30th <a href="/wiki/President_of_the_United_States" title="President

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

In [451]:
# Explanation:

#     Loop through all paragraphs: You loop through the list of paragraphs returned by soup.find_all('p').
#     Condition check: Inside the loop, you check if the paragraph meets your condition. This condition could be a keyword, phrase, or pattern in the text that makes the paragraph unique.
#     Store the text: When the correct paragraph is identified, store its text inside the first_paragraph variable.
#     Exit the loop: Once the right paragraph is found, break the loop to stop further processing.

In [452]:
# <10 lines
for para in paragraphs:
    if para.find('b'): # Check if the paragraph contains any <b> (bold) tags
        first_paragraph = para.get_text().strip()  # Get the text of the first paragraph with bold text
    break # Exit the loop after finding the first match
print(first_paragraph)


Millard Fillmore (January 7, 1800 – March 8, 1874) was the 13th president of the United States, serving from 1850 to 1853, and was the last president to have been a member of the Whig Party while in office. A former member of the U.S. House of Representatives, Fillmore was elected the 12th vice president in 1848, and succeeded to the presidency when Zachary Taylor died in July 1850. Fillmore was instrumental in passing the Compromise of 1850, which led to a brief truce in the battle over the expansion of slavery.


At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [455]:
# 10 lines
# def get_first_paragraph(wikipedia_url):
#   print(wikipedia_url) # keep this for the rest of the notebook
#   [insert your code]
#   return first_paragraph

def get_first_paragraph(paragraphs):
    for para in paragraphs:
        if para.find('b'): # Check if the paragraph contains any <b> (bold) tags
            first_paragraph = para.get_text().strip()  # Get the text of the first paragraph with bold text
            print(first_paragraph)
            return f"First paragraph: {first_paragraph}"



        


In [456]:
# Test: 3 lines

for leader in leaders:
    all_leaders_wiki = leader['wikipedia_url'] # Get the Wikipedia URL for the leader
    response = requests.get(all_leaders_wiki)  # Fetch the page content
    soup = BeautifulSoup(response.text, 'html.parser') # Parse the content    
    # Find all paragraphs in the page
    paragraphs = soup.find_all('p')
    # Loop through the paragraphs to find the first one with bold text
    result = get_first_paragraph(paragraphs)
    print(result)



George Washington (February 22, 1732 – December 14, 1799) was an American Founding Father, politician, military officer, and farmer who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States. Washington has thus become commonly known as the "Father of his Country".
First paragraph: George Washington (February 22, 1732 – December 14, 1799) was an American Founding Father, politician, military officer, and farmer who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of t

### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [372]:
# 3 lines



Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [373]:
# 10 lines


Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [374]:
# < 20 lines


## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [375]:
# < 20 lines

In [376]:
# Check the output of your function (2 lines)


Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [377]:
# < 25 lines



Check the output of your function again.

In [378]:
# Check the output of your function (1 line)


Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [379]:
# < 20 lines


Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [380]:
# <25 lines


Test your new functions.



## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [381]:
# 3 lines


Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [382]:
# 3 lines


Make a function `save(leaders_per_country)` to call this code easily.

In [383]:
# 3 lines


In [384]:
# Call the function (1 line)


## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## To go further

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!